# 古典最適化アルゴリズムを設定する

ここでは、量子古典ハイブリッド計算であるVQEやQAOAで利用される古典最適化アルゴリズムについて、古典最適化アルゴリズムの設定方法をみてみたいと思います。

## Blueqatのインストール
インストールはpipから簡単に行うことができます。

In [3]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 設定方法
古典最適化アルゴリズムは、ansatzの実行の際にminimizerを指定するだけで変更できます。

古典最適化のアルゴリズムは指定できます。主にscipyから選べます。
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP'

In [8]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe, get_scipy_minimizer

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
minimizer = get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(h),minimizer=minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.99999996077787
Result by numpy
-8.0


## その他の最適化ソルバーの実装
その他、自分で使いたい最適化ソルバーを実装するには下記のように関数を用意して、minimizerとして利用します。今回はベイズ最適化を対象として、hyperoptを導入してみます。

In [5]:
!pip install hyperopt

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [6]:
def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)
    return list(best.values())

In [7]:
runner = Vqe(QubitAnsatz(h),minimizer=hyperopt_minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

100%|██████████| 100/100 [00:01<00:00, 88.83trial/s, best loss: -7.995049681622847] 
Result by VQE
-7.995049681622847
Result by numpy
-8.0


このように無事導入ができました。